In [6]:
#!apt install libgtk2.0-dev pkg-config -y


In [7]:
from collections import defaultdict

import cv2
import numpy as np
import ultralytics
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')


In [8]:

# Open the video file
video_path = "./data3/KRA-44-168-2023-08-30-morning.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

In [9]:

# get video dims
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'DIVX')
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('./video9.mp4', fourcc, 20.0, size)


count=1


while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    count += 11 # i.e. at 30 fps, this advances one second
    cap.set(cv2.CAP_PROP_POS_FRAMES, count)

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
            
        out.write(annotated_frame)


    else:
        # Break the loop if the end of the video is reached
        break

cap.release()
out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'



0: 384x640 1 truck, 14.6ms
Speed: 3.2ms preprocess, 14.6ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 15.3ms
Speed: 4.9ms preprocess, 15.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


AttributeError: 'NoneType' object has no attribute 'int'